KoGPT2 파인튜닝 https://github.com/SKT-AI/KoGPT2

### 구글 마운트

In [1]:
# 구글 드라이브 마운트
from google.colab import drive
drive.mount('/content/drive')

# 구글 드라이브 경로 설정
%cd /content/drive/MyDrive/Colab Notebooks/인공지능 학원/프로젝트/텍스트
%pwd

Mounted at /content/drive
/content/drive/MyDrive/Colab Notebooks/인공지능 학원/프로젝트/텍스트


'/content/drive/MyDrive/Colab Notebooks/인공지능 학원/프로젝트/텍스트'

### 라이브러리 설치

In [2]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 18.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 33.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 51.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 53.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 26.5 MB/s eta 0:00:00


### 라이브러리 불러오기

In [3]:
import pandas as pd
# for padding
# from tensorflow.keras.preprocessing.sequence import pad_sequences

import torch
from torch.utils.data import Dataset, DataLoader
from transformers import AdamW
from tqdm import tqdm

import time
import datetime

### 모델 테스트


In [ ]:
from transformers import PreTrainedTokenizerFast
tokenizer = PreTrainedTokenizerFast.from_pretrained("skt/kogpt2-base-v2",
bos_token='</s>', eos_token='</s>', unk_token='<unk>',
pad_token='<pad>', mask_token='<mask>')

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


In [ ]:
tokenizer.tokenize("안녕하세요. 한국어 GPT-2 입니다.😤:)l^o")

['▁안녕',
 '하',
 '세',
 '요.',
 '▁한국어',
 '▁G',
 'P',
 'T',
 '-2',
 '▁입',
 '니다.',
 '😤',
 ':)',
 'l^o']

In [ ]:
import torch
from transformers import GPT2LMHeadModel

model = GPT2LMHeadModel.from_pretrained('skt/kogpt2-base-v2')

In [ ]:
text = '근육이 커지기 위해서는'
input_ids = tokenizer.encode(text, return_tensors='pt')
gen_ids = model.generate(input_ids,
                        max_length=128,
                        repetition_penalty=2.0,
                        pad_token_id=tokenizer.pad_token_id,
                        eos_token_id=tokenizer.eos_token_id,
                        bos_token_id=tokenizer.bos_token_id,
                        use_cache=True)
generated = tokenizer.decode(gen_ids[0])
print(generated)

근육이 커지기 위해서는 무엇보다 규칙적인 생활습관이 중요하다.
특히, 아침식사는 단백질과 비타민이 풍부한 과일과 채소를 많이 섭취하는 것이 좋다.
또한 하루 30분 이상 충분한 수면을 취하는 것도 도움이 된다.
아침 식사를 거르지 않고 규칙적으로 운동을 하면 혈액순환에 도움을 줄 뿐만 아니라 신진대사를 촉진해 체내 노폐물을 배출하고 혈압을 낮춰준다.
운동은 하루에 10분 정도만 하는 게 좋으며 운동 후에는 반드시 스트레칭을 통해 근육량을 늘리고 유연성을 높여야 한다.
운동 후 바로 잠자리에 드는 것은 피해야 하며 특히 아침에 일어나면 몸이 피곤해지기 때문에 무리하게 움직이면 오히려 역효과가 날 수도 있다.
운동을


### 사용자 정의 함수

In [4]:

import re

# 가사 전처리
def preprocess(sentences):
    lst = []
    for sentence in sentences:
        # \n과 \t를 제거한다
        sentence = re.sub('\t', '', sentence)
        sentence = re.sub('\n', '', sentence)
        # 한국어, 공백 빼고 제거하기
        # sentence = re.sub('[^ㄱ-ㅎ가-힣\s]+', '', sentence)
        # 문장 양옆의 띄어쓰기를 지운다
        sentence = sentence.strip()

        lst.append(sentence)
    return lst

### 데이터 불러오기

In [5]:
song_df = pd.read_excel('가사_all.xlsx')
song_df['Lyrics'] = preprocess(song_df['Lyrics'])

In [6]:
song_df['Lyrics']

0       어린 햇살 아래서뛰어놀곤 했었던가쁜 숨결굽이진 골목 지나길을 따라가보면같은 기억어른...
1       꽃이 언제 피는지 그딴 게 뭐가 중요한데날씨가 언제 풀리는지 그딴 거 알면 뭐 할 ...
2       늦게 다니지좀 마술은 멀리좀 해봐열살짜리 애처럼 말을 안듣니정말 웃음만 나와누가 누...
3       난 또 행복한 척 더 더 행복한 척난 또 행복한 척 더 더 행복한 척아무에게도 말하...
4       밥을 먹어도 혼자야구를 봐도 혼자거리를 걸어도 나 혼자혼자이기 싫은데또 방에 누워도...
                              ...                        
3341    잠에서 막 일어난 얼굴도 귀여워잠겨버린 목소리 그것도 귀여워너는 아마 그것까진 모를...
3342    아무말도 없이 날 바라보는게뭔가 이상해서 두귀를 막았어내 눈에 보이는 너의 입모양이...
3343    This is the tihs that y'all ain't ready for Th...
3344    하루 종일 비가 와서니 얼굴이 또 생각 나 눈물이 났어한 걸음 더 걸어가면 잊혀질까...
3345    관둬 가차없이 너랑 끝낼래너같은 남잔 필요없어대체 그럴꺼면 왜 나를 만나니딴 여자랑...
Name: Lyrics, Length: 3346, dtype: object

### DataLoader

In [7]:
lst = ['사랑은 눈에 보이지가 않아어디에 있는지 몰라더듬거리다 찾아다니다이제야 손 끝에 닿았나 봐그때 그대 뒤를 돌아서내게 짓는 웃음에 난 눈을 떠 바람에 흩날리는 그대 향기가낮게 들려오는 그대 목소리가겨울처럼 차가운 내 맘그 안에 스며 들어 와사랑이 눈꽃으로 피어 나그렁 그렁 눈물이 고여이 사랑이 언젠가 시들까 봐바람에 흩날리는 그대 향기가낮게 들려오는 그대 목소리가겨울처럼 차가운 내 맘그 안에 스며 들어 와사랑으로 피어 나하루 이틀 사흘 나흘내 시간을 다 쓸 만큼그대 하나로 난 행복 해바람에 흩날리는 그대 향기가낮게 들려오는 그대 목소리가겨울처럼 차가운 내 맘그 안에 스며 들어 와사랑이 눈꽃으로 피어 나']

In [8]:
len(lst[0]) # 338

338

In [9]:
song_df['Lyrics']

0       어린 햇살 아래서뛰어놀곤 했었던가쁜 숨결굽이진 골목 지나길을 따라가보면같은 기억어른...
1       꽃이 언제 피는지 그딴 게 뭐가 중요한데날씨가 언제 풀리는지 그딴 거 알면 뭐 할 ...
2       늦게 다니지좀 마술은 멀리좀 해봐열살짜리 애처럼 말을 안듣니정말 웃음만 나와누가 누...
3       난 또 행복한 척 더 더 행복한 척난 또 행복한 척 더 더 행복한 척아무에게도 말하...
4       밥을 먹어도 혼자야구를 봐도 혼자거리를 걸어도 나 혼자혼자이기 싫은데또 방에 누워도...
                              ...                        
3341    잠에서 막 일어난 얼굴도 귀여워잠겨버린 목소리 그것도 귀여워너는 아마 그것까진 모를...
3342    아무말도 없이 날 바라보는게뭔가 이상해서 두귀를 막았어내 눈에 보이는 너의 입모양이...
3343    This is the tihs that y'all ain't ready for Th...
3344    하루 종일 비가 와서니 얼굴이 또 생각 나 눈물이 났어한 걸음 더 걸어가면 잊혀질까...
3345    관둬 가차없이 너랑 끝낼래너같은 남잔 필요없어대체 그럴꺼면 왜 나를 만나니딴 여자랑...
Name: Lyrics, Length: 3346, dtype: object

In [10]:
class SongDataset(Dataset):
    def __init__(self, tokenizer, data):
        lst = [str(i) for i in data.values]

        inputs = tokenizer(
            lst,
            return_tensors="pt",
            padding="max_length",
            truncation=True,
            max_length=768,
        )

        self.item = inputs["input_ids"]
        self.attention_mask = inputs["attention_mask"]

        self.length = len(lst)

    def __len__(self):
        return self.length

    def __getitem__(self, idx):
        return self.item[idx], self.attention_mask[idx]

In [11]:
from transformers import PreTrainedTokenizerFast
tokenizer = PreTrainedTokenizerFast.from_pretrained("skt/kogpt2-base-v2",
                bos_token='</s>', eos_token='</s>', unk_token='<unk>',
                pad_token='<pad>', mask_token='<mask>')

print('Lyrics의 최대 길이: ', max([len(lyrics) for lyrics in song_df['Lyrics']]))

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


Lyrics의 최대 길이:  2516


In [12]:
tokenizer.tokenize('가나다')

['▁가', '나다']

In [13]:
batch_size=2

train_data = SongDataset(tokenizer, song_df['Lyrics'])
train_dataloader = DataLoader(train_data, batch_size=batch_size)

In [14]:
# 데이터셋 잘 만들어졌는지 확인
train_data.__getitem__(3)

(tensor([ 9769,  9108, 29918, 11220,  9267,  9267, 29918, 11220,  7058,  9108,
         29918, 11220,  9267,  9267, 29918, 11220,  7965,  7556, 19387, 50521,
         11335, 13103,  8146,  9341,  8146, 10431, 14042,  6889, 12158, 21871,
         30514,  9366, 11525,  8719,  7653,  8711, 11295, 24692,  9049,  6872,
          9176,  7089, 11941,  9689, 11389, 20937,  6958,  7198,  9038, 20849,
         17582,  8146, 10431, 10341,  9566,  8135, 47318,  8006,  7060, 38142,
          9080, 42399,  9136, 18256, 21871, 31187,  6958,  9033,  9135,  9136,
         29500,  8102,  7058,  9108, 29918, 11220,  9267,  9267, 29918, 11220,
          9033, 17582, 16238, 13761, 10056, 13358,  8137,  9068, 11523,  6824,
          9222,  8095,  9068,  6958, 41508, 13358,  8137,  9068, 47588,  6889,
         13358,  8137,  9068,  7058,  9108, 29918, 11220,  9267,  9267, 29918,
         11220,  9033, 17582, 39417, 39502,  9154, 15651, 12371,  9923,  8420,
          8015,  9176,  8146, 12114, 17272, 10099,  

In [15]:
# 데이터로드 잘 만들어졌는지 확인
dataiter = iter(train_dataloader)
item, attention_mask = next(dataiter)

In [16]:
item

tensor([[10452, 15749,  7760,  ...,     3,     3,     3],
        [13146, 12668,  9301,  ...,     3,     3,     3]])

In [17]:
attention_mask

tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])

### GPU 사용

In [18]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print('No GPU available, using the CPU instead.')

There are 1 GPU(s) available.
We will use the GPU: Tesla T4


### 모델 만들기

In [19]:
import torch
from transformers import GPT2LMHeadModel

model = GPT2LMHeadModel.from_pretrained('skt/kogpt2-base-v2').to(device)

In [20]:
model

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(51200, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=51200, bias=False)
)

In [21]:
optimizer = AdamW(model.parameters(),
                  lr = 2e-5,
                  eps = 1e-8)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [22]:
def format_time(elapsed):
    elapsed_rounded = int(round((elapsed)))
    return str(datetime.timedelta(seconds=elapsed_rounded))  # hh:mm:ss

In [23]:
load_data = torch.load("model_kogpt2/kogpt2_fine-tunning_19.pt")

In [24]:
load_data.keys()

dict_keys(['epoch', 'loss', 'model', 'optimizer'])

In [29]:
load_epoch = load_data['epoch']
load_epoch

19

In [26]:
# 저장한 모델 불러오기
# 모델 로드
model.load_state_dict(load_data['model'])

<All keys matched successfully>

In [27]:
# cuda 메모리 지우기
import torch
torch.cuda.empty_cache()

import gc
gc.collect()

237

In [28]:
def generate(keyword):
    input_ids = tokenizer.encode(keyword, return_tensors='pt')
    input_ids = input_ids.to(device)
    gen_ids = model.generate(input_ids,
                        max_length=128,
                        repetition_penalty=2.0,
                        pad_token_id=tokenizer.pad_token_id,
                        eos_token_id=tokenizer.eos_token_id,
                        bos_token_id=tokenizer.bos_token_id,
                        use_cache=True)
    generated = tokenizer.decode(gen_ids[0])
    return generated

In [ ]:
# 학습 전 generate 함수 동작 테스트
keywords = ['밥을 먹어도', '매일같이', '배고파']

for keyword in keywords:
    #keyword = keyword.to(device)
    gen = generate(keyword)
    gen_print = gen.split('\n')[0:2]
    print(f"{keyword} : {gen_print}")

밥을 먹어도 : ['밥을 먹어도 눈물이 나는데 왜 자꾸만 웃음이 나와서 미치도록 아파하는데 사랑해줘 제발 내 손을 잡아봐요 그대 눈을 봐도 눈물방울이 흘러와 가슴이 터질 듯 아파와 난 미칠 것 같아 이별 앞에선 숨을 죽여야 해 그댄 이미 죽었고 나는 벌써 죽어있어 다른 사람 만나면 어떡해야 하는지 어떻게 해야 할까 가슴 아픈 얘기들만 가득하고 내가 미워 죽을 것만 같은데 사랑이 아닌걸 알면서도 이렇게 사는게 너무 아프고 또 슬퍼지는데 행복하게 살아줄 사람은 바로 너인데 나를 떠나버린거라면 정말 미안한 마음 뿐이야 하지만 슬픈 사랑은 멈']
매일같이 : ['매일같이 영화도 보고싶어  니가 좋아하던 그 카페를 갔었지 너와 나 둘만의 데이트 코스는 어디든 상관없고 둘이서 걸으면 딱 좋은데 나도 모르게 어느새 내 모습이 보여 자꾸 눈물이 나서 울어버렸나봐 이젠 말할게 시간이 됐는지 우린 알고 있지만 언제부턴가나만 이런 느낌일까 하는 아쉬움이 앞선거야 내가 먼저 말을 걸어줄테니 조금 더 기다려볼래 우리 함께했던 시간들 모두 기억해보면 너무나 많은 날이 지나버렸지만 아직도 난 네 생각에 잠 못들고 있어 이렇게라도 말해야겠지어쩌면 오늘은 말하고']
배고파 : ['배고파서 잠 못 들어요 그대 생각만 하면 눈물 나죠 그댄 어디 있나요한 번을 말해도 눈물이 날까봐 오늘도 난 울기만 해그대 없이 잘 수 없는데난 아직도 이렇게 살아있어내게 남은 사랑인걸 알잖아세상에 단 한 사람 오직 너뿐인데언젠간 꼭 돌아올 거라고 믿고 있어한번 더 기다려줘야해사랑했던 시간들 모두 흘려버릴테니까이제 행복한 모습으로다시 만날거라 믿어약속할께 다시 한번더 생각해줄꺼란 걸 알아하지만 아직은 너무 늦어너무 보고 싶은데눈물나게']


In [30]:
load_epoch

19

In [31]:
# 학습 모드
model.train()
min_loss = 1

for epoch in range(load_epoch + 1, 15+load_epoch):
    print('======== Epoch {:} / {:} ========'.format(epoch, 15+load_epoch))
    t0 = time.time()
    total_loss = 0

    for input_text, attention_mask in tqdm(train_dataloader):
        input_tensor = input_text.to(device)
        attention_mask = attention_mask.to(device)
        # print(input_tensor)
        # print(input_tensor.shape)

        # input과 label이 같다.
        outputs = model(input_tensor, labels=input_tensor, attention_mask=attention_mask)
        # print('outputs:', outputs)
        loss = outputs[0]

        # cpu로 변경하기
        input_tensor.cpu()

        optimizer.zero_grad()
        model.zero_grad()
        loss.backward()
        optimizer.step()

    now_loss = outputs[0].item()
    print(f"epoch {epoch} loss {now_loss:0.2f}")
    print("  Training epcoh took: {:}".format(format_time(time.time() - t0)))

    # loss가 min_loss보다 작을 경우
    if now_loss < min_loss:
        min_loss = now_loss
        # 모델 저장
        torch.save({'epoch':epoch,
                    'loss':min_loss,
                    'model':model.state_dict(),
                    'optimizer':optimizer.state_dict()},
                   f"model_kogpt2/kogpt2_fine-tunning_{epoch}.pt")

    # 문장 출력해보기
    keywords = ['밥을 먹어도', '매일같이', '배고파', '학원에서']
    for keyword in keywords:
        gen = generate(keyword)
        gen_print = gen.split('\n')[0:2]
        print(f"{keyword} : {gen_print}")


======== Epoch 20 / 34 ========


100%|██████████| 1673/1673 [13:56<00:00,  2.00it/s]


epoch 20 loss 0.73
  Training epcoh took: 0:13:56
밥을 먹어도 : ['밥을 먹어도 눈물이 마를 때까지바라보기도 싫어 눈물조차조금만 더 참아내면 난 어떡해눈물마저 마르기전에그댈 놓아두고영원히 행복하게 해주길바래요 나의 슬픔이제 모두 흘려주세요가슴에 그대 상처까지다 준거죠잊어내고 잊혀져가던 사랑인데왜 떠나야만 하는지죽도록 미워내며 살아왔던 바램과내가 아닌 다른 사람 만나서 미안하다 말하던 그 사람이너무나 보고 싶었네사랑이 너무 많아서 그래 울고 있단 말도 못하고가슴으로 너만을 안을 뿐인걸날']
매일같이 : ['매일같이 혼자있는 하루가이제는 너무 왠지살찌는 것 같아요즘엔 지쳤어네 생각에 또 울었어나를 떠나야 하는게너무 슬퍼져서쓰러질 것만 같던 내 입가에흐르는 눈물 때문에너의 얼굴은 너무나 그리운 얼굴로또 다시 웃음짓죠그렇게 난 너의 얼굴을 잊을 수 없어아닌 척 웃어도 돼난 니 곁에 있어줘언젠간 한번만 널 볼 거예요(내 눈 속에) 영원히안녕이란 말 대신늘 생각해 주겠니널 사랑해 내가 이렇게못견디도록 잘 지내준거라 생각했나봐눈']
배고파 : ['배고파 내 말 들어줄래요내겐 너뿐이야난 네가 필요해이제 그만 보내지 않을게날 도와줘서 고맙다고기다리라고말해줄께언제나 걱정할거야니가 없이는 난 살 수가 없잖아이런 날 이해해주길잘한 거라 생각했어그만둬 기억나면 안돼넌 정말 바보같아너도 나처럼널 사랑한다면부디 내게 말해봐내가 널 너무 모르진 않아조금씩 커져가는 맘 알지만시간이 지나 모두 변해갈수록너의 손길이 나의 것이 될 수 있다면힘들었었던 우리 추억에 조금은']
학원에서 : ["학원에서 만나줄래요나는 행복하댔죠같은 반에서 만났던 친구들처럼행복할 수 있다면그냥 스무살 친데서로를 사랑하면 안돼요한번만 더 나를 바꿔봐줘날 믿어 줬으면난 매일매일 똑같은 하루가너와 내가 함께라면더 좋을텐데이제껏 날 지켜준 너에게금이득 채워질게널 위해 기도해언제나넌 행복한 사람이 돼라that's won't you seet the only for mesave with a

100%|██████████| 1673/1673 [13:55<00:00,  2.00it/s]


epoch 21 loss 0.69
  Training epcoh took: 0:13:55
밥을 먹어도 : ['밥을 먹어도 눈물이 마를 때까지 난 혼자가 아니야  사랑따윈 저버렸는데 왜 나를 죽였나요 정말 미안해 이젠 말할꺼예전 그 모습 그대로인데 하지만 너무 보고싶어 죽을때까지 내곁에 있어줄래 나만 행복할수있도록 살아줘서 고마워세상 모든게 그대 뜻대로 안돼 너무나 두려워질순 없어 그런말 하지 말아봐 내가 너의 전부니까 다른사람 만나도 나는 그렇게 지켜져야만 해 그래놓고 혼자 남겨진 채로 살 순 없는걸 알지만 가슴속에 박힌 슬픈 상처들도 모두 잊을 수 있도록그래 서러울']
매일같이 : ['매일같이 난 그대를 본 적 없어요아주 오래전 우리들의 인연은 여기까지였죠그대 날 떠나던 그날에 눈물짓네요가까이서 지켜줄께난 정말 그대여야만 해요한 순간도 내 맘을 숨기지 않아어차피 기다린 일인데 왜 자꾸만 나를 떠미는지 몰라걱정스런 니 얼굴수많은 추억들 속에사소한 행복에도후회하는 바보같은 나인걸니가 얼마나 예쁜 사람인지아직도 나는 못잊은거야날 울리는 전화벨소리어제 또 누굴 만나러 가야하는지내가 너무 그리워서혹시 그녀가 생각나']
배고파 : ['배고파서 뭐 할까 가사만 꾹 참고 있다가 일어나면 돼 너 오늘 나갈래 오빠나 나 잘 될거야 남자친구 얘기 좀 해줘 나도 여잔데 못되게 굴지마 난 니가 좋은걸어떡하면 좋아 내가 요리조리 다할게그냥 이뻐그러니까 지금부터 손도 안보고 리듬 타기만 추근대끼는 데이트비틀잠깐 참아줄께요우리 둘 사이 매일자꾸 떠오르던 우리사이 멀어지던 시간인 가사빌드림한 요일웃을땐 둘이 편해 우린 남들처럼이쁜 옷 입고 걸터']
학원에서 : ['학원에서 멀어져 버린 너  그땐 나도 저걸 보곤 했어 스무살그때부터 난 내 기억속에 살았었지 하지만 어느새 시간은 나를 떠나갔고 나 혼자가 아닌 걸 존재 자체가 없어졌다 이게 바로 니가 날 피한 이유였었던 거야 내가 널 사랑했던 그만큼 미워했기에 나는 다시 네곳을 밟기 위해 노력해 왔는데 너무나 많은 시간이 흐르고 그녀가 떠난 

...

epoch 22 loss 0.65
  Training epcoh took: 0:13:55
밥을 먹어도 : ['밥을 먹어도 가슴은 아프지 않나요매일 눈을 뜨고 나서 눈뜨면 난 다시 잠들겠죠그런데 왜 이렇게 아픈가요가까부시게 나를 떠나던 그 순간도 내 마음 아팠었어사랑했던 날 잊으셨는지이젠 괜찮아졌단 말밖에 못해줬는데왜 자꾸 울기만 하니난 아직도 미련하게 남아서정말 보고 싶어서눈을 감아야 하는데마치 다른 사람처럼 또 하루를 살아숨쉬기 힘들어져언제라도 볼 수 있게내 눈에 내가 박혀서가슴에 멍들어와상처만 더 남기고그리곤 해서혹']
매일같이 : ['매일같이 밥을 먹다 보면난 딴 사람 있어요자리에 들어설 때면내 손을 잡고있죠아무런 말 없이괜히 울고 싶어졌어요한 시간 동안잠이 들었나 봐요가만한 눈물이 멈추질 않아그냥 이대로 두어줘세상 모든 게 내겐 너무 소중해부탁할게 날 사랑한다면날 받아 줄래몇 번을 다시 물어봐도계속 잘 될 거라고 말해줄께오랜 친구처럼항정 난 너를 기쁘게 해주고 싶어나는 heaven you so hot너와 함께 할 수 있다면노래가 끝나기 전에']
배고파 : ['배고파 서영은 너의 얘기회사 일로 밤을 새워도 네가 없는 게 너무 답답해서잠이 들었어 니 생각만 하면 자꾸 눈물이 나너랑 있는게 어색해그때는 말야 지금 뭐할래 내가 할 수있는 건다 말해줘지금 무슨 말을 해봐 말할게이미안하다는 말은 마이크를 잡고 싶어서 말도 못하고가슴을 떨구던 순간난 너무나 행복했지사랑한다는 맘 서로 가진거라곤 믿었던 거야가끔은 속상한 마음뿐일 때가 있어조금씩 조금 더 잠들기를다시 한 번더 물어볼 걸그래']
학원에서 : ['학원에서 멀었던이 동네는 너무 좋았지  너와 함께 걷던 그 길엔 예쁜 꽃도 피어 있었지기다리는 나의 모습 머리위로 아주리 안녕 그대여 눈을 감고 다시 봐요오래된 목련인걸 니 얼굴을 보며 눈물이 나죠그댄 누구보다 내 사랑에너무 빛나 보이네요가까스로 불러주니 고마워 행복해너만 보면 이렇게 눈물 나는 걸혹시 우린 얼마나 좋은지나근데 왜 자꾸 울게 하는 건지아쉬운 맘에 

100%|██████████| 1673/1673 [13:55<00:00,  2.00it/s]


epoch 23 loss 0.60
  Training epcoh took: 0:13:55
밥을 먹어도 : ['밥을 먹어도 속이 허전해요그냥 먹을 수 없네요가끔 내 맘에 안 좋은데드라마를 봐도 보고싶지 않아 편하게 먹는 것도좋지만약속시간은 잘 졌는데정말 싫은 표정 짓는 내가 너무 싫어맛있는 것 같아 좋았던 일들만 잔뜩하고속이 아프다든지 터진듯한 심장이식탁 위에 올려서도 계속 내려 앉고심장은 점점 멈춰 있는데난 왜 식어버린 건 아닌지나봐 니가 좀 답답해서숨이 막혀 와괜히 눈물이 나나란 사람 참 나쁘더라밥만 해도 가슴이 막혀서 죽겠는데너 때문에']
매일같이 : ["매일같이 영화도 보고싶어 uhBaby part me tonightDrink this day마치 잠든 남자처럼나른하게 잠을 잘 수 있어bodylova di lipsand you wanna flowin' intoxical Dance밤새도록 내 맘을 괴롭혀온 남자들나를 보면 모두 다 멍해지고심장은 멈춰버렸지난 참술을 마셔주면 난숨이 차올라잠에 들어버린 생각들이제발 좀 그만해줘널 위해서 너를 비워두려고 해봐complete여긴"]
배고파 : ['배고파서 싸우다 지친 몸을 쉬다 나만 한심해 눈물이 핑 도네 어느덧 훌쩍훌라 커버린 머리위로 바람이 차오를 때쯤 나도 모르게 다시 한번 쿵짝뛰어 볼까 저기 어디 있는 거냐 놀아볼래 너를 기다리며 하루 일년 내내 기다린 끝에 드디어 내 눈에 들어왔지 널 위한 노래야 물론 다 그래 니가 밥 먹을땐 항상 옆에 있게 돼 난 그게 좋아 이제 곧 부를거야나도 몰래 연습했었던 랩퍼들 전부 귀여워 했었다니 춤 출때는 두벌 하게 될부츠는 따로 있었지어디까지 갔']
학원에서 : ['학원에서 3학년이던 시절내겐 꿈은 없었지난해 겨울날틀린 수학교과서를 졸던 때네 생각이 내 전부였어그때넌 아무 말 없이나와 함께 있었고이제 다른 친구들은 모두밤새도록 괴롭혔지주위를 잃고 자존심도 잃어버렸지오랜만에 날 보며말한마디에 난 웃어버렸고뒤통수만 녀석들아저씨 손엔 잡히는게 없냐고혹시 모르니오래된 연인이 아니야좋

  2%|▏         | 30/1673 [00:15<14:02,  1.95it/s]


KeyboardInterrupt: ignored

In [ ]:
# 학습 후 generate 함수 동작 테스트
keywords = ['밥을 먹어도', '매일같이', '배고파']

for keyword in keywords:
    #keyword = keyword.to(device)
    gen = generate(keyword)
    gen_print = gen.split('\n')[0:2]
    print(f"{keyword} : {gen_print}")

밥을 먹어도 : ['밥을 먹어도 눈물이 나는데 왜 자꾸만 웃음이 나와서 미치도록 아파하는데 사랑해줘 제발 내 손을 잡아봐요 그대 눈을 봐도 눈물방울이 흘러와 가슴이 터질 듯 아파와 난 미칠 것 같아 이별 앞에선 숨을 죽여야 해 그댄 이미 죽었고 나는 벌써 죽어있어 다른 사람 만나면 어떡해야 하는지 어떻게 해야 할까 가슴 아픈 얘기들만 가득하고 내가 미워 죽을 것만 같은데 사랑이 아닌걸 알면서도 이렇게 사는게 너무 아프고 또 슬퍼지는데 행복하게 살아줄 사람은 바로 너인데 나를 떠나버린거라면 정말 미안한 마음 뿐이야 하지만 슬픈 사랑은 멈']
매일같이 : ['매일같이 영화도 보고싶어  니가 좋아하던 그 카페를 갔었지 너와 나 둘만의 데이트 코스는 어디든 상관없고 둘이서 걸으면 딱 좋은데 나도 모르게 어느새 내 모습이 보여 자꾸 눈물이 나서 울어버렸나봐 이젠 말할게 시간이 됐는지 우린 알고 있지만 언제부턴가나만 이런 느낌일까 하는 아쉬움이 앞선거야 내가 먼저 말을 걸어줄테니 조금 더 기다려볼래 우리 함께했던 시간들 모두 기억해보면 너무나 많은 날이 지나버렸지만 아직도 난 네 생각에 잠 못들고 있어 이렇게라도 말해야겠지어쩌면 오늘은 말하고']
배고파 : ['배고파서 잠 못 들어요 그대 생각만 하면 눈물 나죠 그댄 어디 있나요한 번을 말해도 눈물이 날까봐 오늘도 난 울기만 해그대 없이 잘 수 없는데난 아직도 이렇게 살아있어내게 남은 사랑인걸 알잖아세상에 단 한 사람 오직 너뿐인데언젠간 꼭 돌아올 거라고 믿고 있어한번 더 기다려줘야해사랑했던 시간들 모두 흘려버릴테니까이제 행복한 모습으로다시 만날거라 믿어약속할께 다시 한번더 생각해줄꺼란 걸 알아하지만 아직은 너무 늦어너무 보고 싶은데눈물나게']


In [ ]:
keywords = ['나 혼자', '학원에서', '감정과는']

for keyword in keywords:
    #keyword = keyword.to(device)
    gen = generate(keyword)
    gen_print = gen.split('\n')[0:2]
    print(f"{keyword} : {gen_print}")

나 혼자 : ['나 혼자 사랑하고 있어요 난 그댈 위해서 살고있어요한 번만 더 나를 봐줘봐 부탁이야 내 눈물이야 다시는 울지 않을게 약속해 줄래 내가 아파할 수 있게가슴이 터질 것 같은데난 이미 늦었어그대 없이 사는 게 얼마나 힘든지 알고 있잖아사랑하는 사람 곁에 있어도언제나 함께 할 거라 믿었는데날 떠나버린 후 너무나 보고싶은 날이지나간 후에라도기다릴걸 그랬나 저 멀리서 바라볼 걸바랬는데왜 이렇게도 멀어지는 건지아무것도 모르는 채로 살아야만 하는건지나조차 너무 두려워한번더 내게']
학원에서 : ["학원에서 치킨을 먹었어요즘엔 괜히 바쁜가봐요가만해졌나 봐요는 없네 그냥 편하게 앉아있기만 해바텐더 한잔 마실까 말야난 잘 모르겠는데그냥 갈게 난<unk>네가 좋아하던 그 맛 그대로를 원하고 있어you're mine one show yea wonderland new club of downand that's a buckin in the foodwithout likes hots get ready, alrightit’ll taking to kno"]
감정과는 : ["감정과는 달라도 너무 다른 사람표현 속엔 항상 네가 있어그냥 내 방식대로말을 해봐why want to be smile왜 이리도 생생한데 왜 날 아프게 하는지이젠 알 것 같아너무 많이 변했어 너와 나 둘만의 관계stay with me, and give it only you난 말야 내가 필요해너의 맘을 열 수 있게그래서 난 말을 해야 돼With Me Woo, this love foreverYeahtonight, I'll alreadembers these destination of c"]


In [ ]:
epoch

In [ ]:
# 모델 저장
torch.save(model.state_dict(), f"model_kogpt2/kogpt2_fine-tunning_{epoch}.pt")

In [ ]:
# 모델 로드
# model.load_state_dict(torch.load("model_kogpt2/kogpt2_fine-tunning_14.pt"))